In [ ]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib as mplib
import matplotlib.pyplot as plt
import matplotlib.image as img
import itertools
from keras import layers, models
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
from collections import Counter
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('./hmnist_28_28_RGB.csv')
print(data.shape)

In [ ]:
Y = data['label']
print(type(Y))
print(Counter(Y))
nlabels = Y.value_counts().size
print(nlabels)

In [ ]:
X = data.drop(['label'],axis=1)
X = X/255

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
smote = SMOTE(random_state = 50)
X_resample, Y_resample = smote.fit_resample(X, Y)

In [ ]:
print(X_resample.shape)
print(Y_resample.shape)

In [ ]:
unique, counts = np.unique(Y_resample, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
X_resample = X_resample.values.reshape(-1,28,28,3)

In [ ]:
Y_resample = tf.keras.utils.to_categorical(Y_resample, num_classes=nlabels)
print(Y_resample.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_resample, Y_resample, test_size=0.1, random_state=1)

In [ ]:
print(Y_train.shape)
print(X_train.shape)
print(Y_test.shape)
print(X_test.shape)

In [ ]:
#CNN Architecture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 ->
                           #Flatten -> Dense -> Dropout -> Out
model = models.Sequential()


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu', input_shape=(28,28,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu', input_shape = (28,28,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))


model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu', input_shape = (28,28,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same',
                       activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))


model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(nlabels,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
augdata = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,
)  # randomly flip images

augdata.fit(X_train)

In [ ]:
unique, counts = np.unique(Y_resample, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
data1 = model.fit(
    X_train,
    Y_train,
    epochs=10,
    verbose = 1,
    # validation_data = (X_val, Y_val),
#     steps_per_epoch = X_train.shape[0] // 128,
)